In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from bs4.element import Comment
import requests
import urllib.request
import cssutils
import re
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import time

In [2]:
pd.set_option("display.max_columns", 156)

In [3]:
driver=webdriver.Chrome(r"C:\ProgramData\chocolatey\bin\chromedriver")

In [4]:
op_df=pd.DataFrame(pd.read_csv("output.csv"))


In [5]:
op_df

,title,learn_type,topics,skills,description,cover_image,cover_video,embedded_video_url,delivery_method,instruction_type,content,what_will_learn,prerequisites,target_students,instructor|1|name,instructor|1|designation,instructor|1|instructor_bio,instructor|1|linkedin_url,instructor|1|facebook_url,instructor|1|twitter_url,instructor|1|instructor_image,instructor|2|name,instructor|2|designation,instructor|2|instructor_bio,instructor|2|linkedin_url,instructor|2|facebook_url,instructor|2|twitter_url,instructor|2|instructor_image,instructor|3|name,instructor|3|designation,instructor|3|instructor_bio,instructor|3|linkedin_url,instructor|3|facebook_url,instructor|3|twitter_url,instructor|3|instructor_image,instructor|4|name,instructor|4|designation,instructor|4|instructor_bio,instructor|4|linkedin_url,instructor|4|facebook_url,instructor|4|twitter_url,instructor|4|instructor_image,instructor|5|name,instructor|5|designation,instructor|5|instructor_bio,instructor|5|linkedin_url,instructor|5|facebook_url,instructor|5|twitter_url,instructor|5|instructor_image,review|1|reviewer_name,review|1|photo,review|1|review_date,review|1|review,review|1|rating,review|2|reviewer_name,review|2|photo,review|2|review_date,review|2|review,review|2|rating,review|3|reviewer_name,review|3|photo,review|3|review_date,review|3|review,review|3|rating,review|4|reviewer_name,review|4|photo,review|4|review_date,review|4|review,review|5|reviewer_name,review|5|photo,review|5|review_date,review|5|review,review|5|rating,review|5|rating.1,total_duration,total_duration_unit,total_video_content,total_video_content_unit,recommended_effort_per_week,avg_session_duration_with_instructor,batch|1|batch_size,batch|1|batch_start_date,batch|1|batch_end_date,batch|2|batch_size,batch|2|batch_start_date,batch|2|batch_end_date,enrollment_start_date,enrollment_end_date,level,languages,short_description,subtitle_languages,accessibilities,availabilities,Display Price,pricing_type,currency,regular_price,sale_price,additional_pricing_details,course_financing_available,indian_students_program_fee,indian_students_payment_deadline,indian_students_GST_included,indian_student_installments|1|installment_amount,indian_student_installments|1|payment_deadline,indian_student_installments|2|installment_amount,indian_student_installments|2|payment_deadline,international_students_program_fee,international_students_payment_deadline,international_student_installments|1|installment_amount,international_student_installments|1|payment_deadline,international_student_installments|2|installment_amount,international_student_installments|2|payment_deadline,institute,partner_course_url,corporate_sponsor|1|name,corporate_sponsor|1|logo,corporate_sponsor|2|name,corporate_sponsor|2|logo,accreditation|1|name,accreditation|1|logo,accreditation|1|description,accreditation|2|name,accreditation|2|logo,accreditation|2|description,assessment_content,post_course_interaction,international_faculty,human_interaction,personalized_teaching,live_class,job_assistance,internship,alumni_network,placement|1|company_name,placement|1|role_offered,placement|2|company_name,placement|2|role_offered,learning_mediums,virtual_labs,case_based_learning,capstone_project,average_salary,highest_salary,application_seat_ratio,bounce_rate,completion_ratio,enrollment_ratio,faculty_student_ratio,gender_diversity,student_stream_diversity,student_nationality_diversity,average_salary_hike,instructor_citations,syllabus


In [6]:
urls=["https://teamtreehouse.com/library/threads-and-services","https://teamtreehouse.com/library/intents-and-broadcast-receivers","https://teamtreehouse.com/library/unit-testing-in-java","https://teamtreehouse.com/library/testing-in-android","https://teamtreehouse.com/library/android-widgets","https://teamtreehouse.com/library/kotlin-for-java-developers"]

In [7]:
len(urls)

6

In [8]:
html_content=[]
soup=[]

for i in range (len(urls)):
    html_content.append(requests.get(urls[i]).text)
    soup.append(BeautifulSoup(html_content[i], "lxml"))

In [9]:
title=[]
for i in soup:
    title.append(i.title.text)
op_df["title"]=title

In [10]:
description=[]
for i in soup:
    try:
        pt=(i.find(class_="grid-66 tablet-grid-60").find_all("p"))
        description.append(pt[1])
    except:
        print("Error")
op_df["description"]=description

In [11]:
inst_1_name=[]
for i in soup:
    try:
        inst_1_name.append(i.find("div",{"id" : "syllabus-authors"}).find("h4").text)
    except:
        inst_1_name.append(i.find("div",{"id" : "workshop-authors"}).find("h4").text)
op_df["instructor|1|name"]=inst_1_name

In [12]:
inst_bio=[]
for i in soup:
    inst_bio.append(i.find(class_="grid-33 tablet-grid-40").find("p").text)
op_df["instructor|1|instructor_bio"]=inst_bio

In [13]:
inst_desig=[]
for i in soup:
    inst_desig.append("Teacher")
op_df["instructor|1|designation"]=inst_desig

In [14]:
inst_img=[]
for i in soup:
    prt=(i.find(class_="instructor-avatar")["style"].split("("))
    inst_img.append(prt[1].replace(")",""))
op_df["instructor|1|instructor_image"]=inst_img

In [15]:
level=[]
for i in soup:
    try:
        level.append(i.find("li",{"id" : "syllabus-skill-level"}).text)
    except:
        level.append(i.find("li",{"id" : "workshop-skill-level"}).text)
op_df["level"]=level

In [16]:
topics=[]
for i in soup:
        topics.append(i.find("ul",class_="tags actions").find("a").text)
op_df["topics"]=topics

In [17]:
op_df["partner_course_url"]=urls

In [18]:
what_will_learn=[]
for i in soup:
    try:
        what_will_learn.append(i.find(class_="check-list").text.replace("\n","|"))
        
    except:
        what_will_learn.append(i.find(class_="check-list"))
op_df["what_will_learn"]=what_will_learn

In [19]:
learn_type=[]
for i in soup:
    if(i.text.find("Workshop"))>=1:
        learn_type.append("Workshop")
    else:
        learn_type.append("Course")
op_df["learn_type"]=learn_type

In [20]:
duration=[]
duration_unit=[]
for i in soup:
    try:
        pt=(i.find("div",{"id" : "syllabus-title"}).find("h2").text.replace("\n","").split(" "))
        pt=(pt[4].split("-"))
        duration.append(pt[0])
        duration_unit.append(pt[1])
    except:
        pr=(i.find("div",{"id" : "workshop-title"}).find("h2").text.replace("\n","").split(" "))
        pr=(pr[4].split("-"))
        duration.append(pr[0])
        duration_unit.append(pr[1])
                
op_df["total_duration"]=duration
op_df["total_duration_unit"]=duration_unit       

In [21]:
lang=[]
for i in soup:
    lang.append("English")
op_df["languages"]=lang
op_df["subtitle_languages"]=lang

In [22]:
inst_type=[]
for i in soup:
    inst_type.append("Self Paced")
op_df["instruction_type"]=inst_type

In [23]:
available=[]
for i in soup:
    available.append("Downloadable Content")
op_df["availabilities"]=available

In [25]:
content=[]
for i in urls:
    tempro=[]
    driver.get(i)
    time.sleep(5)
    a=driver.find_elements_by_class_name("toggle-steps")
    for i in a:
        time.sleep(2)
        try:
            i.click()
        except:
            pass
    pri=(driver.find_elements_by_class_name("stage-meta"))
    for j in pri:
        prnt=(j.text.replace(":","").replace("questions","").replace("\n","|"))
        res = ''.join([i for i in prnt if not i.isdigit()])
        tempro.append(res)
    content.append(tempro)

op_df["content"]=content

In [26]:
op_df #17 Kpis (Excluding Content)

,title,learn_type,topics,skills,description,cover_image,cover_video,embedded_video_url,delivery_method,instruction_type,content,what_will_learn,prerequisites,target_students,instructor|1|name,instructor|1|designation,instructor|1|instructor_bio,instructor|1|linkedin_url,instructor|1|facebook_url,instructor|1|twitter_url,instructor|1|instructor_image,instructor|2|name,instructor|2|designation,instructor|2|instructor_bio,instructor|2|linkedin_url,instructor|2|facebook_url,instructor|2|twitter_url,instructor|2|instructor_image,instructor|3|name,instructor|3|designation,instructor|3|instructor_bio,instructor|3|linkedin_url,instructor|3|facebook_url,instructor|3|twitter_url,instructor|3|instructor_image,instructor|4|name,instructor|4|designation,instructor|4|instructor_bio,instructor|4|linkedin_url,instructor|4|facebook_url,instructor|4|twitter_url,instructor|4|instructor_image,instructor|5|name,instructor|5|designation,instructor|5|instructor_bio,instructor|5|linkedin_url,instructor|5|facebook_url,instructor|5|twitter_url,instructor|5|instructor_image,review|1|reviewer_name,review|1|photo,review|1|review_date,review|1|review,review|1|rating,review|2|reviewer_name,review|2|photo,review|2|review_date,review|2|review,review|2|rating,review|3|reviewer_name,review|3|photo,review|3|review_date,review|3|review,review|3|rating,review|4|reviewer_name,review|4|photo,review|4|review_date,review|4|review,review|5|reviewer_name,review|5|photo,review|5|review_date,review|5|review,review|5|rating,review|5|rating.1,total_duration,total_duration_unit,total_video_content,total_video_content_unit,recommended_effort_per_week,avg_session_duration_with_instructor,batch|1|batch_size,batch|1|batch_start_date,batch|1|batch_end_date,batch|2|batch_size,batch|2|batch_start_date,batch|2|batch_end_date,enrollment_start_date,enrollment_end_date,level,languages,short_description,subtitle_languages,accessibilities,availabilities,Display Price,pricing_type,currency,regular_price,sale_price,additional_pricing_details,course_financing_available,indian_students_program_fee,indian_students_payment_deadline,indian_students_GST_included,indian_student_installments|1|installment_amount,indian_student_installments|1|payment_deadline,indian_student_installments|2|installment_amount,indian_student_installments|2|payment_deadline,international_students_program_fee,international_students_payment_deadline,international_student_installments|1|installment_amount,international_student_installments|1|payment_deadline,international_student_installments|2|installment_amount,international_student_installments|2|payment_deadline,institute,partner_course_url,corporate_sponsor|1|name,corporate_sponsor|1|logo,corporate_sponsor|2|name,corporate_sponsor|2|logo,accreditation|1|name,accreditation|1|logo,accreditation|1|description,accreditation|2|name,accreditation|2|logo,accreditation|2|description,assessment_content,post_course_interaction,international_faculty,human_interaction,personalized_teaching,live_class,job_assistance,internship,alumni_network,placement|1|company_name,placement|1|role_offered,placement|2|company_name,placement|2|role_offered,learning_mediums,virtual_labs,case_based_learning,capstone_project,average_salary,highest_salary,application_seat_ratio,bounce_rate,completion_ratio,enrollment_ratio,faculty_student_ratio,gender_diversity,student_stream_diversity,student_nationality_diversity,average_salary_hike,instructor_citations,syllabus
0,Threads and Services Course (How To) | Treehouse,Course,Android,NaN,"[Learn about Services in Android, which can pe...",NaN,NaN,NaN,NaN,Self Paced,[Threads in Android|Every piece of code in And...,None,NaN,NaN,Ben Deitch,Teacher,Ben is an Android teacher at Treehouse with a ...,NaN,NaN,NaN,'https://secure.gravatar.com/avatar/d74a55f8bc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [28]:
op_df.to_excel("Advanced_Android.xlsx")